# MSPrior
**MSPrior** is a multi(scale/stream) prior model for realtime temporal learning using RAVE models created by Antoine Caillon. 
\
[Source code on Github](https://github.com/caillonantoine/msprior)
\
\
**RAVE** is a variational autoencoder for fast and high-quality neural audio synthesis by Antoine Caillon and Philippe Esling.
\
[Article on arxiv](https://arxiv.org/abs/2111.05011) & [Source code on Github](https://github.com/acids-ircam/RAVE)

----

Notebook Author: [Martin Heinze](https://github.com/devstermarts) 

Date: 24.10.2023

## Install Conda, MSPrior, dependencies

This installs all neccessary components and libraries to run this notebook on Kaggle. \
\
*Note: the current version (24.10.2023) of MSPrior training stops at epoch 1000. In order to run the training beyond 1000 epochs, the setup script below downloads a different train.py from devstermarts' fork of MSPrior where the training does not stop automatically. The training cell after the preprocessing step contains command lines for both, the default training as well as the alternative version.*

In [ ]:
#Install Conda
!mkdir /kaggle/temp
%cd /kaggle/temp
!curl -L https://repo.anaconda.com/miniconda/Miniconda3-py39_4.12.0-Linux-x86_64.sh -o miniconda.sh
!chmod +x miniconda.sh
!sh miniconda.sh -b -p /kaggle/temp/miniconda

#Upgrade ipython ipykernel, install ffmpeg
!/kaggle/temp/miniconda/bin/pip install --upgrade ipython ipykernel
!/kaggle/temp/miniconda/bin/conda install ffmpeg --yes

#Install MSPRIOR 
#Pin to specific release tag if necessary using 'acids-msprior==tag.version‘
!/kaggle/temp/miniconda/bin/pip install acids-msprior==1.1.2

#Force Torch Reinstall with adequate CUDA driver version
#!/kaggle/temp/miniconda/bin/pip install torch==2.1.0 torchvision==0.16.0 torchaudio==2.1.0 --index-url https://download.pytorch.org/whl/cu118
!/kaggle/temp/miniconda/bin/pip install torch torchvision torchaudio --force-reinstall --index-url https://download.pytorch.org/whl/cu118

#Download devstermarts train.py (to train > 1000 epochs)
!mkdir /kaggle/temp/msprior/
!mkdir /kaggle/temp/msprior/msprior_scripts/
!curl -L https://raw.githubusercontent.com/devstermarts/msprior/master/msprior_scripts/train.py -o /kaggle/temp/msprior/msprior_scripts/train.py

## Preprocess the dataset

Use the filepath to your audio dataset for the --audio flag. This is usually /kaggle/input/your_audio_dataset when you include a dataset uploaded to Kaggle. \
Also add the path of your previously trained RAVE model after the --rave flag. Note that the RAVE model needs to be one exported without the --streaming flag. \
On completion of preprocessing, you will find files lock.mdb, data.mdb and the model in the /kaggle/working/processed folder. \
For all possible flags and parameters of preprocessing, check preprocess.py in https://github.com/caillonantoine/msprior/tree/master/msprior/msprior_scripts \
\
*Note: Preprocessing is neccessary only once per training. When resuming training, make sure to disable this section of the notebook.*

In [ ]:
!mkdir /kaggle/working/processed

%cd /kaggle/working/
!/kaggle/temp/miniconda/bin/msprior preprocess \
--audio /kaggle/input/your_audio_dataset \
--out_path /kaggle/working/processed \
--rave /kaggle/input/your_rave_model_without_streaming.ts

## Start training

Options for --config are: decoder_only / recurrent / encoder_decoder / encoder_decoder_continuous \
*Note: the last two are dependent on rave2vec, which has not yet been open sourced (24.10.2023)* \
Training progress will be stored in /kaggle/working/runs/name_of_your_training \
For all possible parameters, check train.py in https://github.com/caillonantoine/msprior/tree/master/msprior_scripts and the .gin file to your chosen configuration in https://github.com/caillonantoine/msprior/tree/master/msprior/config \
\
*Note: When resuming training, make sure to disable this section of the notebook.*

In [ ]:
%cd /kaggle/working/
#!/kaggle/temp/miniconda/bin/msprior train \
!/kaggle/temp/miniconda/bin/python /kaggle/temp/msprior/msprior_scripts/train.py \
--db_path /kaggle/working/processed/ \
--name name_of_your_training \
--config recurrent

## Resume training
In order to conveniently resume training RAVE on Kaggle, you can transform the output data of the earlier run into a new dataset from the output tab of your notebook. You then add the new dataset to the notebook when editing (or update dataset, if it has been integrated into the notebook already), disable the above initial, **Preprocess the dataset** and **Start training** sections of the notebook and adjust the below section according to your earlier training details. \
\
*Note: The following section is disabled by default, enable when resuming.*  

In [ ]:
##Copy contents of earlier training to /kaggle/working folder.
#!cp -r /kaggle/input/your_earlier_training_run/* /kaggle/working

#%cd /kaggle/working/
##!/kaggle/temp/miniconda/bin/msprior train \
#!/kaggle/temp/miniconda/bin/python /kaggle/temp/msprior/msprior_scripts/train.py \
#--db_path /kaggle/working/processed/ \
#--name name_of_your_training \
#--config recurrent \
#--ckpt /kaggle/working/runs/name-of-your-training/version_X/checkpoints/checkpoint-file.ckpt

## Export model
After your training is finished, you can export a model (.ts Torchscript) file. \
Use the --continuous flag if you trained MSPrior with any but a discrete RAVE model. \
The --batch_size=64 flag fixes a current export issue with models trained with recurrent configuration, which has not yet been adressed (24.10.2023). \
The exported model will be created in /kaggle/working/runs/name_of_your_training/ \
For all possible parameters, check export.py in https://github.com/caillonantoine/msprior/tree/master/msprior/msprior_scripts

In [ ]:
#Copy contents of earlier training to /kaggle/working folder.
!cp -r /kaggle/input/your_earlier_training_run/* /kaggle/working

%cd /kaggle/working/
!/kaggle/temp/miniconda/bin/msprior export \
--run /kaggle/working/runs/name_of_your_training \
--continuous \
--batch_size=64